In [3]:
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]


translation_model_name = "Helsinki-NLP/opus-mt-ru-en"
translation_model = MarianMTModel.from_pretrained(translation_model_name)
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)

inputs = translation_tokenizer(financial_text_russian, return_tensors="pt", padding=True, truncation=True)
outputs = translation_model.generate(**inputs)
translated_text = translation_tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("Translated Text:", translated_text)

# Choose the FinBERT model and tokenizer for classification
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Tokenize the translated text for FinBERT
inputs = finbert_tokenizer(translated_text, padding = True, truncation = True, return_tensors='pt')

# Make predictions with FinBERT
outputs = finbert_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# print(predictions)


positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline': financial_text_russian,
         'En Headline': translated_text,
         "Positive": positive,
         "Negative": negative, 
         "Neutral": neutral}
      
df = pd.DataFrame(table, columns=["Headline", 'En Headline', "Positive", "Negative", "Neutral"])

df['real_score'] =  df['Positive'] - df['Negative']

df

Translated Text: ['Company X has announced record profits for the last quarter.', 'Company X announced record losses over the last quarter.', 'Company X has no change.', 'The electric motor startup of the Arrival ex-head Yota will leave Russia.', 'Schroeder declined an offer to join the Gazprom Board of Directors.', "Shelf is being developed / Gazproma's contractor at sea may be Andrei Patrushev's company.", 'The net profit of RusHidro on ASHI increased by 17% in the first half of the year', 'The Finnish tyre manufacturer Nokian Tyres decided to leave Russia.', 'Fedun retired as Vice President of Lucoil.', 'Promsbank has strengthened its position in the top 10 of Russian banks on the loan portfolio to organizations.']


,Headline,En Headline,Positive,Negative,Neutral,real_score
0,Компания X объявила о рекордных прибылях за по...,Company X has announced record profits for the...,0.948517,0.025460,0.026023,0.923057
1,Компания X объявила о рекордных убытках за пос...,Company X announced record losses over the las...,0.009817,0.972690,0.017493,-0.962874
2,Компания X имеет показатели без изменений.,Company X has no change.,0.024528,0.030658,0.944815,-0.006130
3,Электромобильный стартап Arrival экс-главы Yot...,The electric motor startup of the Arrival ex-h...,0.035713,0.033495,0.930791,0.002218
4,Шрёдер отклонил предложение войти в совет дире...,Schroeder declined an offer to join the Gazpro...,0.043275,0.024247,0.932478,0.019028
5,Шельф берут в разработку // Генподрядчиком «Га...,Shelf is being developed / Gazproma's contract...,0.033618,0.024472,0.941910,0.009145
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,The net profit of RusHidro on ASHI increased b...,0.954949,0.014268,0.030784,0.940681
7,Финский производитель шин Nokian Tyres решил у...,The Finnish tyre manufacturer Nokian Tyres dec...,0.016718,0.811226,0.172056,-0.794508
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,Fedun retired as Vice President of Lucoil.,0.019642,0.112515,0.867843,-0.092873
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,Promsbank has strengthened its position in the...,0.959419,0.017457,0.023124,0.941961


In [6]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]

rubert_tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
rubert_model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)
# Tokenize the text for RuBERT
inputs = rubert_tokenizer(financial_text_russian, padding=True, truncation=True, return_tensors='pt')

# Make predictions with RuBERT
outputs = rubert_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)



# Process and display the predictions
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline': financial_text_russian, "Positive": positive, "Negative": negative, "Neutral": neutral}
df = pd.DataFrame(table, columns=["Headline", "Positive", "Negative", "Neutral"])

df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Headline,Positive,Negative,Neutral
0,Компания X объявила о рекордных прибылях за по...,0.806638,0.160117,0.033245
1,Компания X объявила о рекордных убытках за пос...,0.804880,0.159613,0.035507
2,Компания X имеет показатели без изменений.,0.806738,0.152314,0.040948
3,Электромобильный стартап Arrival экс-главы Yot...,0.816962,0.151060,0.031978
4,Шрёдер отклонил предложение войти в совет дире...,0.180748,0.067817,0.751434
5,Шельф берут в разработку // Генподрядчиком «Га...,0.182502,0.068359,0.749138
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,0.820529,0.147761,0.031711
7,Финский производитель шин Nokian Tyres решил у...,0.814953,0.140481,0.044566
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,0.802255,0.136789,0.060956
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,0.819028,0.149560,0.031412


In [7]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]


tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews', return_dict=True)

inputs = tokenizer(financial_text_russian, padding=True, truncation=True, return_tensors='pt')

# Make predictions with RuBERT
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)



# Process and display the predictions
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline': financial_text_russian, "Positive": positive, "Negative": negative, "Neutral": neutral}
df = pd.DataFrame(table, columns=["Headline", "Positive", "Negative", "Neutral"])

df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Headline,Positive,Negative,Neutral
0,Компания X объявила о рекордных прибылях за по...,0.456038,0.317209,0.226753
1,Компания X объявила о рекордных убытках за пос...,0.450040,0.071163,0.478797
2,Компания X имеет показатели без изменений.,0.405029,0.491894,0.103077
3,Электромобильный стартап Arrival экс-главы Yot...,0.433272,0.024305,0.542423
4,Шрёдер отклонил предложение войти в совет дире...,0.416257,0.020574,0.563170
5,Шельф берут в разработку // Генподрядчиком «Га...,0.578657,0.185546,0.235798
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,0.482959,0.051739,0.465301
7,Финский производитель шин Nokian Tyres решил у...,0.440304,0.017323,0.542373
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,0.309432,0.031819,0.658749
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,0.469166,0.307080,0.223754


In [9]:
import torch
from transformers import pipeline

# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]

model = pipeline(model="seara/rubert-tiny2-russian-sentiment")

inputs = tokenizer(financial_text_russian, padding=True, truncation=True, return_tensors='pt')

# Make predictions with RuBERT
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

print(predictions)

# # Process and display the predictions
# positive = predictions[:, 0].tolist()
# negative = predictions[:, 1].tolist()
# neutral = predictions[:, 2].tolist()

# table = {'Headline': financial_text_russian, "Positive": positive, "Negative": negative, "Neutral": neutral}
# df = pd.DataFrame(table, columns=["Headline", "Positive", "Negative", "Neutral"])

# df

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[9.9995e-01, 1.2872e-32, 8.3887e-07,  ..., 1.2289e-13,
          1.8071e-24, 6.0730e-19],
         [9.2518e-18, 1.3231e-31, 7.9704e-09,  ..., 3.3550e-19,
          9.6623e-26, 6.3849e-22],
         [4.3035e-10, 1.3441e-20, 4.5170e-06,  ..., 5.4596e-10,
          1.0978e-15, 1.5624e-13],
         ...,
         [9.8436e-11, 4.7983e-25, 9.9990e-01,  ..., 6.3069e-13,
          2.8260e-19, 3.1897e-16],
         [9.8436e-11, 4.7983e-25, 9.9990e-01,  ..., 6.3069e-13,
          2.8260e-19, 3.1897e-16],
         [9.8436e-11, 4.7983e-25, 9.9990e-01,  ..., 6.3069e-13,
          2.8260e-19, 3.1897e-16]],

        [[9.9996e-01, 1.0820e-32, 8.8729e-07,  ..., 1.1589e-13,
          1.6191e-24, 5.3940e-19],
         [3.6229e-17, 5.4038e-31, 1.8917e-08,  ..., 9.4380e-19,
          3.6174e-25, 2.3733e-21],
         [4.4677e-10, 9.5940e-21, 5.0669e-06,  ..., 6.4483e-10,
          1.0514e-15, 1.5622e-13],
         ...,
         [8.2228e-11, 1.7737e-25, 9.9991e-01,  ..., 1.8367e-13,
          9.282

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch


# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]


from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

inputs = tokenizer(financial_text_russian, padding=True, truncation=True, return_tensors='pt')
outputs = model(**inputs)

# Example: Assuming a linear layer for classification on top of the output embeddings
classifier = torch.nn.Linear(model.config.hidden_size, 3)  # num_classes is the number of classes for classification
logits = classifier(outputs.last_hidden_state.mean(dim=1))  # Mean pooling over the tokens

# For classification, you might directly use the logits or apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)

print(probabilities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[0.2222, 0.2808, 0.4970],
        [0.2220, 0.2892, 0.4888],
        [0.2358, 0.2867, 0.4775],
        [0.2167, 0.3386, 0.4447],
        [0.2499, 0.3359, 0.4142],
        [0.2413, 0.3026, 0.4561],
        [0.2154, 0.2507, 0.5338],
        [0.3062, 0.2860, 0.4078],
        [0.2904, 0.3361, 0.3735],
        [0.2040, 0.2842, 0.5118]], grad_fn=<SoftmaxBackward0>)
